In [1]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))

In [2]:
proj <- loadArchRProject("../data/VisiumHeart", showLogo = FALSE)
metadata <- as.data.frame(proj@cellColData)

Successfully loaded ArchRProject!



In [3]:
counts <- readRDS("../data/VisiumHeart/PeakMatrix.Rds")
dim(counts)

[1] 319939  17700

In [4]:
chrom_assay <- CreateChromatinAssay(
  counts = counts,
  sep = c("_", "_"),
  genome = 'hg38',
  min.cells = 30
)

obj.atac <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks",
  meta.data = metadata,
  names.field = 1, 
  names.delim = "#")

obj.atac

An object of class Seurat 
319834 features across 17700 samples within 1 assay 
Active assay: peaks (319834 features, 0 variable features)

In [7]:
library(reticulate)
sc <- import("scopen")
matDR <- sc$Main$scopen_dr(counts = obj.atac@assays$peaks@counts,
                           verbose = 1)

matDR <- t(as.matrix(matDR))

colnames(matDR) <- paste0("PC_", 1:ncol(matDR))
rownames(matDR) <- colnames(obj.atac@assays$peaks@counts)
obj.atac@reductions[['scopen']] <- CreateDimReducObject(embeddings = matDR,
                                                        assay = "peaks",
                                                       key = "PC_")
DepthCor(obj.atac, reduction = "scopen", n = 30)

Harmony 1/10

Harmony 2/10

Harmony 3/10

Harmony converged after 3 iterations



In [ ]:
obj.atac <- RunUMAP(object = obj.atac, 
                    reduction = 'scopen', 
                    dims = 1:30, 
                    verbose = FALSE)

suppressMessages(library(harmony))

obj.atac <- RunHarmony(
  object = obj.atac,
  group.by.vars = "Sample",
  reduction = 'scopen',
  project.dim = FALSE,
  assay.use = 'peaks',
  plot_convergence = FALSE,
  verbose = TRUE
    
)

In [ ]:
obj.atac <- RunUMAP(obj.atac, 
               dims = 1:30, 
               reduction = 'harmony',
               reduction.name = "umap_harmony",
               reduction.ke = 'umapharmony_',
              verbose = FALSE,
                   min.dist = 0.4)

options(repr.plot.width = 12, repr.plot.height = 5)

p1 <- DimPlot(object = obj.atac, reduction = "umap",
              group.by = "Sample") +
    xlab("UMAP1") + ylab("UMAP2")

p2 <- DimPlot(object = obj.atac, reduction = "umap_harmony",
              group.by = "Sample", shuffle = TRUE) +
    xlab("UMAP1") + ylab("UMAP2")

p1 + p2

In [ ]:
saveRDS(obj.atac, file = "../data/VisiumHeart/snATAC.Rds")

In [ ]:
sessionInfo()